# Intro to Tensorflow

In [4]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

In [76]:
%cd '/Users/Vincent/Desktop/Python/DataCamp/Data'

/Users/Vincent/Desktop/Python/DataCamp/Data


# Loss minimization

In [29]:
# Linear Regression loss minimization
housing = pd.read_csv("kc_house_data.csv")
size_log = np.log(housing['sqft_lot'])
price_log = np.log(housing['price'])
bedrooms = housing['bedrooms']

# Define a linear regression model
def linear_regression(intercept, slope, features):
	return intercept + slope*features

# Set loss_function() to take the variables as arguments
def loss_function(intercept, slope, features, targets):
	# Set the predicted values
	predictions = linear_regression(intercept, slope, features)
    
    # Return the mean squared error loss
	return keras.losses.mse(targets,predictions)

# Initialize an adam optimizer
opt = keras.optimizers.Adam(0.5)

slope, intercept = tf.Variable(5, dtype=float), tf.Variable(0.01, dtype=float)

for j in range(100):
	# Apply minimize, pass the loss function, and supply the variables
	opt.minimize(lambda: loss_function(intercept, slope, features=size_log, targets=price_log)
                 , var_list=[intercept, slope])

	# Print every 20th value of the loss
	if j % 20 == 0:
		print(loss_function(intercept, slope,features= size_log, targets=price_log).numpy())

740.7025
5.9827175
8.582166
2.826127
2.6676488


Loss minimal at last iteration

In [23]:
# Multiple linear regression loss minimization
def linear_regression(params, feature1 = size_log, feature2 = bedrooms):
	return params[0] + feature1*params[1] + feature2*params[2]

# Define the loss function
def loss_function_multiple(params, targets = price_log, feature1 = size_log, feature2 = bedrooms):
	# Set the predicted values
	predictions = linear_regression(params, feature1, feature2)
  
	# Use the mean absolute error loss
	return keras.losses.mae(targets, predictions)

def print_results(params):
    return print('loss: {:0.3f}'\
                 ', intercept: {:0.3f}'\
                 ', slope_1: {:0.3f}'\
                 ', slope_2: {:0.3f}'.format(loss_function(params).numpy()
                                             , params[0].numpy(), params[1].numpy()
                                             , params[2].numpy()))


# Define the optimize operation
opt = keras.optimizers.Adam()

params = tf.Variable([0.1 , 0.05, 0.02], dtype=float)

# Perform minimization and print trainable variables
for j in range(5):
	opt.minimize(lambda: loss_function_multiple(params), var_list=[params])
	print_results(params)

loss: 12.430, intercept: 0.101, slope_1: 0.050, slope_2: 0.020
loss: 12.429, intercept: 0.102, slope_1: 0.050, slope_2: 0.020
loss: 12.428, intercept: 0.103, slope_1: 0.050, slope_2: 0.020
loss: 12.427, intercept: 0.104, slope_1: 0.050, slope_2: 0.020
loss: 12.426, intercept: 0.105, slope_1: 0.050, slope_2: 0.020


# Batch training

In [30]:
# Initialize adam optimizer
opt = keras.optimizers.Adam()

# Load data in batches
for batch in pd.read_csv('kc_house_data.csv', chunksize=100):
	size_batch = np.array(batch['sqft_lot'], np.float32)

	# Extract the price values for the current batch
	price_batch = np.array(batch['price'], np.float32)

	# Complete the loss, fill in the variable list, and minimize
	opt.minimize(lambda: loss_function(intercept, slope, price_batch, size_batch)
                 , var_list=[intercept, slope])

# Print trained parameters
print(intercept.numpy(), slope.numpy())

-2.680509 1.5169338


# Neural Networks

In [107]:
# Construct neural net with one hidden layer from scratch
# for a single observation for now
bias = tf.Variable(1.0)
weights = tf.Variable(tf.ones((3, 2)))

# 1 single observation for now
features = np.array([ 2.,  1., 24.],dtype='float32').reshape(1,-1)

product = tf.matmul(features, weights)
activation = keras.activations.sigmoid(product + bias)

# Initialize bias2 and weights2
bias_hidden = tf.Variable(1.0)
weights_hidden = tf.Variable(tf.ones((2, 1)))

# Perform matrix multiplication of dense1 and weights2
product_hidden = tf.matmul(activation,weights_hidden)

# Apply activation to product2 + bias2 and print the prediction
prediction = keras.activations.sigmoid(product_hidden + bias_hidden)
print('\n prediction: {}'.format(prediction.numpy()[0,0]))
print('\n actual: 1')

# Print the shapes of borrower_features, weights1, bias1, and dense1
print('\n shape of features: ', features.shape)
print('\n shape of weights: ', weights.shape)
print('\n shape of bias: ', bias.shape)
print('\n shape of activation layer: ', activation.shape)


 prediction: 0.9525741338729858

 actual: 1

 shape of features:  (1, 3)

 shape of weights:  (3, 2)

 shape of bias:  ()

 shape of activation layer:  (1, 2)


In [106]:
# Classification
credit = pd.read_csv('uci_credit_card.csv')
bill_amounts = np.array(credit.loc[:,['BILL_AMT1','BILL_AMT2','BILL_AMT3']])
default = np.array(credit.iloc[:,-1]).reshape(-1,1)

# Construct input layer from features
inputs = tf.constant(bill_amounts, dtype='float32')

# first dense layer
dense1 = keras.layers.Dense(3, activation='relu')(inputs)
# second dense layer
dense2 = keras.layers.Dense(2, activation='relu')(dense1)
# output layer
outputs = keras.layers.Dense(1, activation='sigmoid')(dense2)

# Print error for first five examples
error = default[:5] - outputs.numpy()[:5]
print("Errors: \n", error)

Errors: 
 [[ 0.5]
 [ 0.5]
 [-0.5]
 [-0.5]
 [-0.5]]


In [112]:
# Muticlass classification
# Construct input layer from borrower features
features = np.array(credit.iloc[:,12:-1])

inputs = tf.constant(features, dtype='float32')

#  first dense layer
dense1 = keras.layers.Dense(10, activation='sigmoid')(inputs)
#  second dense layer
dense2 = keras.layers.Dense(8, activation='relu')(dense1)
#  output layer
outputs = keras.layers.Dense(6, activation='softmax')(dense2)

# Print first five predictions
print(outputs.numpy()[:5])

[[0.26324698 0.05705299 0.20492303 0.06891117 0.17117931 0.23468648]
 [0.35041356 0.08097418 0.12154561 0.06995234 0.23805432 0.13905996]
 [0.26676628 0.08445688 0.11434913 0.08605982 0.30729672 0.1410712 ]
 [0.2402384  0.06995337 0.12828635 0.07499182 0.27226552 0.21426457]
 [0.27779093 0.0625894  0.10677237 0.05452311 0.2997077  0.19861647]]


Notice that each row of outputs sums to one since a row contains the predicted class probabilities for one example.

# Optimization

In [121]:
import math
pi = 3.141592653589793

def loss_function(x):
    return 4.0*math.cos(x-1) + tf.divide(math.cos(2.0*pi*x),x)

# Initialize x_1 and x_2
x_1 = tf.Variable(0.05,dtype='float32')
x_2 = tf.Variable(0.05,dtype='float32')

# Define the optimization operation for opt_1 and opt_2
opt_1 = keras.optimizers.RMSprop(learning_rate=0.01, momentum=0.99)
opt_2 = keras.optimizers.RMSprop(learning_rate=0.01, momentum=0.00)

for j in range(100):
	opt_1.minimize(lambda: loss_function(x_1), var_list=[x_1])
    # Define the minimization operation for opt_2
	opt_2.minimize(lambda: loss_function(x_2), var_list=[x_2])

# Print x_1 and x_2 as numpy arrays
print(x_1.numpy(), x_2.numpy())

2.7445114 0.24999999


As can be seen below, the optimization without momentum gets stuck in the local minimum of the energy function

![Minima](minima.png)

# Training & Testing

In [152]:
default = np.array(credit.iloc[:,-1], dtype="float32").reshape(-1,1)
features = np.array(credit.iloc[:,1:-1], dtype='float32')

x=np.c_[features,default]
np.random.shuffle(x)

ratio = 2/3
train_features = x[0:int(ratio*len(x))][:,:-1]
test_features = x[int(ratio*len(x)):len(x)][:,:-1]
train_labels = x[0:int(ratio*len(x))][:,-1]
test_labels = x[int(ratio*len(x)):len(x)][:,-1]

In [165]:
# Define the model
def model(w1, b1, w2, b2, features = train_features):
	# Apply relu activation functions to layer 1
	layer1 = keras.activations.relu(tf.matmul(features, w1) + b1)
    # Apply dropout
	dropout = keras.layers.Dropout(0.25)(layer1)
	return keras.activations.sigmoid(tf.matmul(dropout, w2) + b2)

# Define the loss function
def loss_function(w1, b1, w2, b2, features = train_features, targets = train_labels):
	predictions = model(w1, b1, w2, b2)
	# Pass targets and predictions to the cross entropy loss
	return keras.losses.binary_crossentropy(targets, predictions)

# Define the layer 1 weights
w1, b1 = tf.Variable(tf.random.normal([23, 7])), tf.Variable(tf.ones([7]))
# Define the layer 2 weights
w2, b2 = tf.Variable(tf.random.normal([7, 1])), tf.Variable(0.0)

# Train the model
for j in range(100):
	opt.minimize(lambda: loss_function(w1, b1, w2, b2), 
                 var_list=[w1, b1, w2, b2])

# Make predictions and evaluate
model_predictions = model(w1, b1, w2, b2, test_features)

In [166]:
def accuracy(yp,y):
    
    y=y.reshape(-1,1)
    e=yp-y
    o=np.count_nonzero(e)
    
    return ((len(y)-o)/len(y))*100
accuracy(model_predictions,test_labels)

78.03999999999999

Model is about 78% accurate in its prediction